In [6]:
from osgeo import gdal
import numpy as np

In [7]:
path_img = "E:\\YNU\\5\\OpenSourceGIS\\Act_5\\OnClass\\chenggong_utm48.tif"

In [8]:
dset = gdal.Open(path_img) # 打开影像文件
geo_trans = dset.GetGeoTransform() # 获取影像的地理变换参数
print('打印地理变换参数', geo_trans) # 打印地理变换参数
x_min, y_max = geo_trans[0], geo_trans[3] # 获取左上角坐标
x_res, y_res = geo_trans[1], geo_trans[5] # 获取影像的分辨率
print('打印影像的分辨率', x_res, y_res) # 打印影像的分辨率
x_max, y_min = x_min + x_res * dset.RasterXSize, y_max + y_res * dset.RasterYSize # 获取右下角坐标
print('打印左上角和右下角坐标', (x_min, y_max), (x_max, y_min)) # 打印左上角和右下角坐标
print('extent', (x_min, x_max, y_min, y_max)) # 打印extent


打印地理变换参数 (268167.02525260474, 10.421454790528236, 0.0, 2765453.1872716597, 0.0, -9.543567381150961)
打印影像的分辨率 10.421454790528236 -9.543567381150961
打印左上角和右下角坐标 (268167.02525260474, 2765453.1872716597) (298128.7077753734, 2733205.4730907506)
extent (268167.02525260474, 298128.7077753734, 2733205.4730907506, 2765453.1872716597)


In [9]:
x_min_subs, x_max_subs = 281209, 284218
y_min_subs, y_max_subs = 2746702, 2748732

##### 求裁剪区域的左上角的行号和列号

In [10]:
col_start_subs = (x_min_subs - x_min) / x_res
row_start_subs = (y_max_subs - y_max) / y_res
print(col_start_subs, row_start_subs)
# int()向下取整, round()四舍五入取整
col_start_subs_update, row_start_subs_update = round(col_start_subs), round(row_start_subs)
print(col_start_subs_update, row_start_subs_update)

1251.45433238829 1752.0898217457905
1251 1752


##### 更新裁剪区域左上角角点坐标

In [11]:
x_min_subs_update = x_min + col_start_subs_update * x_res
y_max_subs_update = y_max + row_start_subs_update * y_res
print((x_min_subs_update, y_max_subs_update))

(281204.2651955556, 2748732.8572198832)


In [12]:
x_size_subs = (x_max_subs - x_min_subs_update) / x_res
y_size_subs = (y_min_subs - y_max_subs_update) / y_res
print(x_size_subs, y_size_subs)

289.18561419884685 212.7985415489693


In [13]:
x_size_subs_update, y_size_subs_update = round(x_size_subs), round(y_size_subs)
print(x_size_subs_update, y_size_subs_update)

289 213


##### 更新裁剪区域右下角角点坐标

In [14]:
x_max_subs_update = x_min_subs_update + x_size_subs_update * x_res
y_min_subs_update = y_max_subs_update + y_size_subs_update * y_res
print(x_max_subs_update, y_min_subs_update)

284216.06563001825 2746700.0773676983


##### 准确的裁剪空间范围

In [15]:
exten_subs_update = [x_min_subs_update, x_max_subs_update, y_min_subs_update, y_max_subs_update]
print(exten_subs_update)

[281204.2651955556, 284216.06563001825, 2746700.0773676983, 2748732.8572198832]


##### 求裁剪后的地理转换参数

In [16]:
geotrans_subs = [x_min_subs_update, x_res, 0, y_max_subs_update, 0, y_res]
print(geotrans_subs)

[281204.2651955556, 10.421454790528236, 0, 2748732.8572198832, 0, -9.543567381150961]


In [17]:
img_arry = dset.ReadAsArray()
img_arry.shape

(6, 3379, 2875)

In [18]:
img_arry_subs = img_arry[:, 
                         row_start_subs_update:row_start_subs_update + y_size_subs_update,
                         col_start_subs_update:col_start_subs_update + x_size_subs_update]
print(img_arry_subs.shape)

(6, 213, 289)


In [19]:
path_subs = "E:\\YNU\\5\\OpenSourceGIS\\Act_5\\OnClass\\chenggong_wgs84_subs.tif"
driver = gdal.GetDriverByName("GTiff")
dset_subs = driver.Create(path_subs, 
                          xsize = x_size_subs_update,
                          ysize = y_size_subs_update, 
                          bands = dset.RasterCount, 
                          eType = gdal.GDT_Int16)
dset_subs.SetGeoTransform(geotrans_subs)
dset_subs.SetProjection(dset.GetProjection())
for i in range(dset.RasterCount):
    outband = dset_subs.GetRasterBand(i+1)
    outband.WriteArray(img_arry_subs[i])

dset_subs = None